In [1]:
# Проверял, есть ли еще какое-то влияние случайности

# при перезапуске одной модели могут быть незначительные отличия в значениях тотал ревардв в запусках с авариями.

# сид работает нормально. Если заного инициализировать среду и задавать сид, или просто задавать сид и делать резет - результат одинаковый

# для модели обученной на коммите среды 3862ab3b6f6b169712ab07711c1368068e2509f5 сравнивал результаты запусков \
# на этом коммите и на ea3e81a7fb23ee9b4d8946170158772fd4755ce0. Результат практически одинаков. Иногда встречаются отличия +-1, совсем редко +-1.
# только для сравнения этих коммитов надо не забыть поменять множительв if self.finish_position_framestimer > self.frames_per_step * 20:

In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/continuous-grid-arctic/")
sys.path.append("/s/ls4/users/grartem/RL_robots/RL_robotSim/")
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [2]:
os.chdir("/s/ls4/users/grartem/RL_robots/RL_robotSim/")

In [3]:
import time
import gym
import pyhocon
import numpy as np
import pandas as pd
import ray
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.ddpg.td3 import TD3Trainer

import MyMisc

/s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Couldn't import `requests` library. Be sure to install it on the client side.
Traceback (most recent call last):
  File "/s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/ray/tune/web_server.py", line 16, in <module>
    import requests  # `requests` is not part of stdlib.
  File "/s/ls4/users/grartem/.local/lib/python3.8/site-packages/requests/__init__.py", line 43, in <module>
    import urllib3
ModuleNotFoundError: No module named 'urllib3'
Couldn't import `requests` library. Be sure to install it on the client side.


pygame 2.1.2 (SDL 2.0.16, Python 3.8.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
configs = pyhocon.ConfigFactory.parse_file("/s/ls4/users/grartem/RL_robots/RL_robotSim/Configs/FollowerContinuous/PPO_obst.conf")
#CONFIG = configs["ppo_archv3"].as_plain_ordered_dict()
CONFIG = configs["ppo_env4feats10v7_train5v2"].as_plain_ordered_dict()
CONFIG["config"]["num_workers"]=1

In [5]:
CONFIG["config"]["env_config"]

OrderedDict([('name', 'Test-Cont-Env-Auto-v0'),
             ('base_env_config',
              OrderedDict([('add_obstacles', True),
                           ('warm_start', 0),
                           ('max_steps', 10000),
                           ('framerate', 5000),
                           ('obstacle_number', 35),
                           ('constant_follower_speed', False),
                           ('early_stopping',
                            OrderedDict([('max_distance_coef', 1.4),
                                         ('low_reward', -300)])),
                           ('random_frames_per_step', [2, 20]),
                           ('leader_speed_regime',
                            OrderedDict([('0', [0.2, 1]),
                                         ('200', 1),
                                         ('1000', [0.5, 1]),
                                         ('1500', 0.75),
                                         ('2000', 0),
                              

# Chose checkpoint

In [9]:
EXPERIMENT_DIR = "/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous_11042022_env5483ef7a5c89448cd46564db5dc50c7e8f57ca5e//obst_dynLSpd_dynFPS/PPO/env4feats10v7_train/ppo_env4feats10v7_train5v2/PPO_continuous-grid_0a7bf_00000_0_2022-04-12_15-49-50/"
df= pd.read_csv(EXPERIMENT_DIR + "/progress.csv")
df = df.loc[list(range(0,800,10))]


df.sort_values(by=["episode_reward_max", "episode_reward_mean", "episode_reward_min"], ascending= False)
print("top max rewards")
print(df.sort_values(by=["episode_reward_max", "episode_reward_mean", "episode_reward_min"], ascending= False).index[:10])
df.sort_values(by=["episode_reward_mean", "episode_reward_max", "episode_reward_min"], ascending= False)
print("top mean rewards")
print(df.sort_values(by=["episode_reward_mean", "episode_reward_max", "episode_reward_min"], ascending= False).index[:10])
df.sort_values(by=[ "episode_reward_min","episode_reward_mean", "episode_reward_max"], ascending= False)
print("top min rewards")
print(df.sort_values(by=[ "episode_reward_min","episode_reward_mean", "episode_reward_max"], ascending= False).index[:10])

df["reward_sum"] = df.loc[:, ["episode_reward_min","episode_reward_mean", "episode_reward_max"]].sum(axis=1)
df.sort_values(by="reward_sum", ascending= False)

top max rewards
Int64Index([360, 90, 290, 600, 220, 140, 350, 410, 310, 770], dtype='int64')
top mean rewards
Int64Index([180, 290, 260, 660, 760, 120, 640, 300, 190, 310], dtype='int64')
top min rewards
Int64Index([510, 760, 250, 420, 160, 490, 170, 440, 730, 10], dtype='int64')


,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,info/learner/default_policy/learner_stats/cur_kl_coeff,info/learner/default_policy/learner_stats/cur_lr,info/learner/default_policy/learner_stats/total_loss,info/learner/default_policy/learner_stats/policy_loss,info/learner/default_policy/learner_stats/vf_loss,info/learner/default_policy/learner_stats/vf_explained_var,info/learner/default_policy/learner_stats/kl,info/learner/default_policy/learner_stats/entropy,info/learner/default_policy/learner_stats/entropy_coeff,reward_sum
250,915.0,120.0,448.846000,481.100000,11,4,1506000,0,1506000,False,...,0.253125,0.00005,626.196226,-0.012880,626.206158,-0.196721,0.011643,-0.248056,0.0,1483.846000
160,938.0,84.0,455.109000,476.860000,13,4,966000,0,966000,False,...,0.112500,0.00005,712.840029,-0.017776,712.855675,-0.056694,0.018922,0.206500,0.0,1477.109000
760,872.3,121.3,469.075000,503.780000,13,4,4566000,0,4566000,False,...,0.684209,0.00005,532.108109,0.002521,532.096558,0.071104,0.013196,-0.440858,0.0,1462.675000
420,882.0,111.6,451.231000,487.200000,13,4,2526000,0,2526000,False,...,0.427148,0.00005,602.689220,0.005645,602.679687,-0.004970,0.009103,-0.506231,0.0,1444.831000
290,971.0,-13.0,474.894000,502.270000,14,4,1746000,0,1746000,False,...,0.379688,0.00005,552.054796,-0.025625,552.075680,-0.096345,0.012484,-0.319053,0.0,1432.894000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,786.0,-56.0,344.602000,401.190000,16,4,186000,0,186000,False,...,0.050000,0.00005,961.820044,-0.002625,961.822047,-0.018103,0.012466,1.960836,0.0,1074.602000
20,792.0,-51.0,301.439000,367.600000,21,4,126000,0,126000,False,...,0.050000,0.00005,1309.714397,-0.001386,1309.714874,0.202461,0.018276,2.319682,0.0,1042.439000
40,717.5,-57.0,368.716000,440.710000,15,4,246000,0,246000,False,...,0.050000,0.00005,1102.529957,-0.017569,1102.547042,-0.063579,0.009695,1.802133,0.0,1029.216000
10,444.0,-1.0,133.605000,202.760000,23,4,66000,0,66000,False,...,0.050000,0.00005,1168.317985,0.001090,1168.316364,0.149329,0.010705,2.538617,0.0,576.605000


In [7]:
checkpoint_path = EXPERIMENT_DIR + "/checkpoint_000600/checkpoint-600"
table_save_path = EXPERIMENT_DIR + "/checkpoint-600_test_2.csv"

In [8]:
trainer = ray.rllib.agents.registry.get_trainer_class(CONFIG["run"])(CONFIG["config"])
trainer.restore(checkpoint_path)

env = MyMisc.continuous_env_maker(config=CONFIG["config"]["env_config"])

(pid=30703) 2022-05-18 12:44:31,406	ERROR web_server.py:19 -- Couldn't import `requests` library. Be sure to install it on the client side.
(pid=30703) Traceback (most recent call last):
(pid=30703)   File "/s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/ray/tune/web_server.py", line 16, in <module>
(pid=30703)     import requests  # `requests` is not part of stdlib.
(pid=30703)   File "/s/ls4/users/grartem/.local/lib/python3.8/site-packages/requests/__init__.py", line 43, in <module>
(pid=30703)     import urllib3
(pid=30703) ModuleNotFoundError: No module named 'urllib3'
(pid=30703) 2022-05-18 12:44:32,026	WARNING policy_client.py:28 -- Couldn't import `requests` library. Be sure to install it on the client side.


(RolloutWorker pid=30703) pygame 2.1.2 (SDL 2.0.16, Python 3.8.0)
(RolloutWorker pid=30703) Hello from the pygame community. https://www.pygame.org/contribute.html


(RolloutWorker pid=30703) /s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/follow_the_leader_continuous_env.py:279: UserWarning: Одновременно заданы и random_frames_per_step и frames_per_step, будет использоваться random_frames_per_step
(RolloutWorker pid=30703)   warn(
(RolloutWorker pid=30703) /s/ls4/users/grartem/anaconda3/envs/rl_robots/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=30703)   logger.warn(


(RolloutWorker pid=30703) ===Запуск симуляции номер 0===


(RolloutWorker pid=30703) Failed to create secure directory (/run/user/1556/pulse): No such file or directory
(RolloutWorker pid=30703) ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
(RolloutWorker pid=30703) ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
(RolloutWorker pid=30703) ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
(RolloutWorker pid=30703) ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
(RolloutWorker pid=30703) ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
(RolloutWorker pid=30703) ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
(RolloutWorker pid=30703) ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
(RolloutWorker pid=30703) ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default
2022-05-1

# collect test table

In [10]:
history = []
for seed_i in range(100):
    print(seed_i)
    env.seed(seed_i)
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    start_time = time.time()
    while not done:
        # Compute a single action, given the current observation
        # from the environment.
        action = trainer.compute_single_action(obs, explore=False)
        # Apply the computed action in the environment.
        obs, reward, done, info = env.step(action)
        # Sum up rewards for reporting purposes.
        total_reward += reward
    elapsed_time = time.time() - start_time
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print(elapsed_time)
    history.append({
        "seed": seed_i,
        "reward": total_reward,
        "done": done,
        "time": elapsed_time
    })
    history[-1].update(info)
    pd.DataFrame(history).to_csv(table_save_path, index=False, sep=";", encoding="cp1251")
    # Report results.

0
===Запуск симуляции номер 0===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
00:00:15
1
===Запуск симуляции номер 1===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default
/s/ls4/users/grartem/RL_robots/continuous_grid_arctic/continuous_grid_arctic/utils/misc.py:63: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(v.dot(w) / (np.linalg.norm(v, axis=1) * np.linalg.norm(w)))


00:00:10
2
===Запуск симуляции номер 2===


...

АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:31
15
===Запуск симуляции номер 15===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:19
16
===Запуск симуляции номер 16===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


KeyboardInterrupt: 

# analyze test table

In [11]:
import pandas as pd
df_1 = pd.read_csv("/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4/PPO_continuous-grid_8e7ad_00000_0_2022-04-07_15-16-30/checkpoint-380_test.csv", sep=";", encoding="cp1251")
print("total of success or finished by time", len(df_1[df_1["mission_status"].isin(["success", "finished_by_time"])]))
print("total of too far or low reward stops", len(df_1[df_1["agent_status"].isin(["too_far_from_leader", "low_reward"])]))
print("total follower crash", len(df_1[df_1["agent_status"].isin(["crash"])]))
print("total of leader finished", len(df_1[df_1["leader_status"].isin(["finished"])]))
print("sum of rewards", df_1["reward"].sum())
print("sum time", pd.to_timedelta(df_1["time"]).sum())

total of success or finished by time 89
total of too far or low reward stops 8
total follower crash 2
total of leader finished 88
sum of rewards 46195.0
sum time 0 days 00:50:11


In [12]:
import pandas as pd
df_1 = pd.read_csv("/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/feats/ppo_env4_feats10v7/PPO_continuous-grid_d1217_00000_0_2022-04-11_13-26-15/checkpoint-170_test.csv", sep=";", encoding="cp1251")
print("total of success or finished by time", len(df_1[df_1["mission_status"].isin(["success", "finished_by_time"])]))
print("total of too far or low reward stops", len(df_1[df_1["agent_status"].isin(["too_far_from_leader", "low_reward"])]))
print("total follower crash", len(df_1[df_1["agent_status"].isin(["crash"])]))
print("sum of rewards", df_1["reward"].sum())
print("sum time", pd.to_timedelta(df_1["time"]).sum())

total of success or finished by time 55
total of too far or low reward stops 34
total follower crash 10
sum of rewards 45855.80000000004
sum time 0 days 01:16:40


In [13]:
import pandas as pd
df_1 = pd.read_csv("/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/env4feats10v7_train/ppo_env4feats10v7_train5v2/PPO_continuous-grid_0a7bf_00000_0_2022-04-12_15-49-50/checkpoint-700_test_3.csv", sep=";", encoding="cp1251")
print("total of success or finished by time", len(df_1[df_1["mission_status"].isin(["success", "finished_by_time"])]))
print("total of too far or low reward stops", len(df_1[df_1["agent_status"].isin(["too_far_from_leader", "low_reward"])]))
print("total follower crash", len(df_1[df_1["agent_status"].isin(["crash"])]))
print("sum of rewards", df_1["reward"].sum())
print("sum time", pd.to_timedelta(df_1["time"]).sum())

total of success or finished by time 68
total of too far or low reward stops 21
total follower crash 9
sum of rewards 44330.30000000003
sum time 0 days 01:13:19


# record some videos

In [35]:
import pygame
history = []
for repeat in range(10):
    seed_i = 25
    print(seed_i)
    env.seed(seed_i)
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    start_time = time.time()
    #env.env.env._show_tick()
    for object_to_show in env.env.env.game_object_list:
        if object_to_show.name in ["leader", "follower"]:
            cur_image = object_to_show.image
            if hasattr(object_to_show, "direction"):
                cur_image = env.env.env.rotate_object(object_to_show)

    while not done:        
        # Compute a single action, given the current observation
        # from the environment.
        action = trainer.compute_single_action(obs, explore=False)
        # Apply the computed action in the environment.
        obs, reward, done, info = env.step(action)
        # Sum up rewards for reporting purposes.
        total_reward += reward
        #env.env.env._show_tick()
        for object_to_show in env.env.env.game_object_list:
            if object_to_show.name in ["leader", "follower"]:
                cur_image = object_to_show.image
                if hasattr(object_to_show, "direction"):
                    cur_image = env.env.env.rotate_object(object_to_show)
    elapsed_time = time.time() - start_time
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print(elapsed_time)
    history.append({
        "repeat": repeat,
        "reward": total_reward,
        "done": done,
        "time": elapsed_time
    })
    history[-1].update(info)
historyDF_2 = pd.DataFrame(history)

25
===Запуск симуляции номер 52===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:44
25
===Запуск симуляции номер 53===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:44
25
===Запуск симуляции номер 54===


...

АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:45
25
===Запуск симуляции номер 60===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:42
25
===Запуск симуляции номер 61===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
00:00:45


In [28]:
historyDF_2

,seed,reward,done,time,mission_status,agent_status,leader_status
0,25,862.5,True,00:01:25,success,finished,finished
1,25,862.5,True,00:01:24,success,finished,finished
2,25,862.5,True,00:01:25,success,finished,finished
3,25,862.5,True,00:01:27,success,finished,finished
4,25,862.0,True,00:01:25,success,finished,finished
5,25,862.0,True,00:01:26,success,finished,finished
6,25,862.5,True,00:01:26,success,finished,finished
7,25,862.0,True,00:01:23,success,finished,finished
8,25,862.5,True,00:01:24,success,finished,finished
9,25,862.0,True,00:01:26,success,finished,finished


In [30]:
historyDF_2

,repeat,reward,done,time,mission_status,agent_status,leader_status
0,0,476.0,True,00:00:46,fail,crash,moving
1,1,481.0,True,00:00:45,fail,crash,moving
2,2,483.0,True,00:00:45,in_progress,moving,moving
3,3,483.0,True,00:00:45,in_progress,moving,moving
4,4,483.0,True,00:00:43,in_progress,moving,moving
5,5,483.0,True,00:00:45,in_progress,moving,moving
6,6,483.0,True,00:00:44,in_progress,moving,moving
7,7,483.0,True,00:00:44,in_progress,moving,moving
8,8,483.0,True,00:00:44,in_progress,moving,moving
9,9,483.0,True,00:00:44,in_progress,moving,moving


In [36]:
historyDF_2

,repeat,reward,done,time,mission_status,agent_status,leader_status
0,0,483.0,True,00:00:44,in_progress,moving,moving
1,1,483.0,True,00:00:44,in_progress,moving,moving
2,2,483.0,True,00:00:42,in_progress,moving,moving
3,3,483.0,True,00:00:43,in_progress,moving,moving
4,4,483.0,True,00:00:43,in_progress,moving,moving
5,5,483.0,True,00:00:45,in_progress,moving,moving
6,6,483.0,True,00:00:43,in_progress,moving,moving
7,7,483.0,True,00:00:45,in_progress,moving,moving
8,8,483.0,True,00:00:42,in_progress,moving,moving
9,9,483.0,True,00:00:45,in_progress,moving,moving


In [11]:
monitor_env = gym.wrappers.Monitor(env=env,
                                directory= os.path.dirname(checkpoint_path)+"/videos_5",
                               video_callable=lambda _:True,
                               force=False,
                              uid="seeds_test", mode="evaluation")
#for seed_i in [9, 35, 36, 38, 44, 47, 59, 89, 91, 97]:
#for seed_i in [15, 83, 90]:
#for seed_i in [0, 2, 5, 9, 14, 22, 25, 28, 32, 34,36,  38, 42, 47, 52, 54, 58, 61, 62, 64, 75,78, 81, 88,  93, 95]:
for repeat in range(5):
    for seed_i in [25]:
        monitor_env.seed(seed_i)
        obs = monitor_env.reset()
        done = False
        total_reward = 0.0
        # Play one episode.
        start_time = time.time()
        while not done:
            # Compute a single action, given the current observation
            # from the environment.
            action = trainer.compute_single_action(obs, explore=False)
            # Apply the computed action in the environment.
            obs, reward, done, info = monitor_env.step(action)
            # Sum up rewards for reporting purposes.
            total_reward += reward
obs = monitor_env.reset()

===Запуск симуляции номер 0===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


===Запуск симуляции номер 1===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


===Запуск симуляции номер 2===


...

Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
===Запуск симуляции номер 4===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
АВАРИЯ! -10.0
===Запуск симуляции номер 5===


Failed to create secure directory (/run/user/1556/pulse): No such file or directory
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4568:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5047:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2565:(snd_pcm_open_noupdate) Unknown PCM default


In [17]:
print(EXPERIMENT_DIR)

/s/ls4/users/grartem/RL_robots/RL_robotSim/results/FollowerContinuous/obst_dynLSpd_dynFPS/PPO/env4feats10v7_train/ppo_env4feats10v7_train5v2/PPO_continuous-grid_0a7bf_00000_0_2022-04-12_15-49-50/
